# <u>Health Analysis</u>

### Import the necessary Libraries

In [1]:
import pandas as pd
from datetime import datetime
today = datetime.today()

In [2]:
# Load the dataset
df1 = pd.read_excel("Assessment_Data_Set.xlsx", "Dataset")
df2 = pd.read_excel("Assessment_Data_Set.xlsx", "Patient Details")

In [3]:
df1.head()

,VisitDate,PatientId,Location,Payor,InvoiceTotal
0,2019-07-21,PAT24869,Kahawa West,Cash,1350.0
1,2019-05-28,PAT17185,Embakasi,Cash,600.0
2,2019-12-29,PAT19895,Zimmerman,Cash,140.0
3,2019-07-03,PAT20790,Pipeline,Cash,325.0
4,2019-09-22,PAT12119,Huruma,Cash,300.0


In [4]:
df2.head()

,PatientId,DateOfBirth,Gender
0,PAT16851,2012-07-02,Male
1,PAT21149,2020-05-01,Male
2,PAT12467,2009-12-25,Female
3,PAT23784,2011-11-26,Male
4,PAT16344,2020-06-25,Male


### Join the two Dataframes.

In [5]:
# Add the Gender, PatientId and DateOfBirth
df1 = df1.join(df2.set_index("PatientId"), on="PatientId")

# Use join as shown below
'''
df1 = df1.set_index('PatientId').join(df2.set_index('PatientId'))
'''
# you can also use map as shown below
"""
df1['Gender'] = df1.PatientId.map(df2.set_index('PatientId')['Gender'].to_dict())
"""

"\ndf1['Gender'] = df1.PatientId.map(df2.set_index('PatientId')['Gender'].to_dict())\n"

### Calculate the Age

In [6]:
# Convert the date into pandas datetime
df1["DateOfBirth"] = pd.to_datetime(df1.DateOfBirth)

In [7]:
df1['Age'] = df1['DateOfBirth'].apply(lambda x:
                                today.year - x.year -
                                ((today.month, today.day) < (x.month, x.day)))

In [8]:
df1.head(10)

,VisitDate,PatientId,Location,Payor,InvoiceTotal,DateOfBirth,Gender,Age
0,2019-07-21,PAT24869,Kahawa West,Cash,1350.0,2019-07-21,Male,1
1,2019-05-28,PAT17185,Embakasi,Cash,600.0,2019-05-28,Female,1
2,2019-12-29,PAT19895,Zimmerman,Cash,140.0,2019-12-29,Female,1
3,2019-07-03,PAT20790,Pipeline,Cash,325.0,2019-07-03,Male,1
4,2019-09-22,PAT12119,Huruma,Cash,300.0,2019-09-22,Male,1
5,2019-12-25,PAT19868,Githurai 45,Cash,0.0,2019-12-25,Female,1
6,2019-04-06,PAT18541,Kayole,Cash,50.0,2019-04-06,Male,1
7,2019-02-22,PAT14342,Zimmerman,Cash,630.0,2019-02-22,Female,1
8,2019-01-23,PAT16627,Embakasi,Cash,1200.0,2019-01-23,Female,2
9,2019-09-22,PAT21352,Embakasi,Cash,575.0,2019-09-22,Female,1


## Qn. For cash visits, what is the average age of female patients who visited between the start of February and the end of April in 2019? 

##### Filter to get the cash Payments and Female Gender

In [9]:
cash_payments_by_females = df1[(df1.Payor == "Cash") & (df1.Gender == "Female")]

In [10]:
len(cash_payments_by_females)

33045

##### Drop Duplicate rows where of PatientIds to only get the unique Patients.

In [11]:
females = cash_payments_by_females.drop_duplicates(subset=['PatientId'], keep='first')

In [12]:
len(females)

7439

##### Filter to get the dates between Feb and April

In [13]:
females = females[(females['VisitDate'] >= '2019-02-01') 
                                  & (females['VisitDate'] <= '2019-04-30')]

In [14]:
females.describe()

,InvoiceTotal,Age
count,3386.000000,3386.000000
mean,610.944194,22.554932
std,696.009271,17.973740
min,0.000000,1.000000
25%,130.000000,9.000000
50%,350.000000,17.000000
75%,870.000000,33.000000
max,8800.000000,84.000000


The mean age is: 22.02

### Qn. What is the total cost to Penda Health of all cash visits by males in October 2019? Assume an average gross margin of 45% per visit.

Gross margin is a company's net sales revenue minus its cost of goods sold (COGS). In other words, it is the sales revenue a company retains after incurring the direct costs associated with producing the goods it sells, and the services it provides.

##### Filter cash visits by Males

In [15]:
cash_payments_by_male = df1[(df1.Payor == "Cash") & (df1.Gender == "Male")]

##### Filter to get the October data

In [16]:
males = cash_payments_by_male[(cash_payments_by_male['VisitDate'] >= '2019-10-01') 
                              & (cash_payments_by_male['VisitDate'] <= '2019-10-31')]

In [17]:
males.head()

,VisitDate,PatientId,Location,Payor,InvoiceTotal,DateOfBirth,Gender,Age
153,2019-10-30,PAT15909,Embakasi,Cash,1000.0,2019-10-17,Male,1
165,2019-10-15,PAT13034,Zimmerman,Cash,200.0,2019-09-29,Male,1
180,2019-10-12,PAT22765,Kangemi,Cash,0.0,2019-09-25,Male,1
218,2019-10-05,PAT21608,Zimmerman,Cash,1280.0,2019-09-14,Male,1
227,2019-10-28,PAT20982,Umoja,Cash,200.0,2019-10-06,Male,1


In [18]:
males.InvoiceTotal.sum()

1513065.5

In [19]:
55/100 * 1513065.5

832186.025

The Total Cost is therefore: 832186.025

## Qn. On average, how much were female patients between the ages of 18-35 (including 35 year-olds) invoiced per visit in 2019?

In [20]:
total_females = df1[df1.Gender == "Female"]

##### filter to get ages between 18 and 35

In [21]:
middle_aged_females = total_females[(total_females['Age'] >= 18) & (total_females['Age'] <= 35)]

##### The average invoice total for the females

In [22]:
middle_aged_females.describe()

,InvoiceTotal,Age
count,11536.000000,11536.000000
mean,1076.010587,25.046897
std,1374.726930,5.097316
min,0.000000,18.000000
25%,200.000000,21.000000
50%,600.000000,25.000000
75%,1500.000000,29.000000
max,26600.000000,35.000000


On Average Females between 18 and 35 years were invoiced: 1076

## What percentage of visits are by female children between the ages of 0-5 (including 5 year-olds) in 2019?

In [23]:
total_females = df1[df1.Gender == "Female"]

##### The total number of females

In [24]:
len(total_females)

44058

##### Filter to get the female children between 0 and 5yrs

In [25]:
children = total_females[(total_females['Age'] >= 0) & (total_females['Age'] <= 5)]

In [26]:
children.sample(10)

,VisitDate,PatientId,Location,Payor,InvoiceTotal,DateOfBirth,Gender,Age
11099,2019-11-01,PAT12082,Embakasi,CIC Photocard,1980.0,2016-04-13,Female,4
3569,2019-08-20,PAT23240,Githurai 45,Madison,3308.0,2018-08-27,Female,2
794,2019-04-14,PAT11580,Embakasi,Cash,1450.0,2019-01-27,Female,2
4643,2019-06-30,PAT22643,Umoja,Cash,490.0,2018-03-08,Female,2
3215,2019-05-20,PAT18420,Tassia,Cash,150.0,2018-07-03,Female,2
8899,2019-07-22,PAT16134,Huruma,Resolution,3626.0,2016-10-05,Female,4
8157,2019-02-04,PAT22639,Githurai 44,Cash,300.0,2016-07-21,Female,4
12343,2019-03-29,PAT10550,Huruma,CIC Photocard,1180.0,2015-03-27,Female,5
6894,2019-08-14,PAT15845,Kahawa West,Cash,450.0,2017-07-10,Female,3
912,2019-07-04,PAT15990,Huruma,Cash,560.0,2019-04-06,Female,1


##### Percentage of Female Children

In [27]:
(len(children)/len(total_females)) * 100

17.1660084434155

The percentage visits by female children is 17.16%

##### Please consider the following hypothetical scenario (you may need to research the meaning of the healthcare terms used):

We are involved in a capitation agreement with one private company (Sunshine Ltd.), where we provide medical care to their employees based on a capitated fee. Another company, TechLab Ltd., also want to go into a capitation agreement with us in order to provide healthcare to their employees. 

Sunshine Ltd. is paying us Sh2000 per person per year, and have 200 people signed up on the scheme with us. 
TechLab Ltd. are offering us Sh1500 per person per year, and have 500 people that they would like to sign up with us.

Please use the data provided to evaluate TechLab Ltd.’s proposal, and answer the following questions.

Remember to round all answers down to the nearest integer.

### How much revenue could we expect to make from TechLab Ltd.’s offer in a one year period? *

In [28]:
1500*500

750000

### How many times can we expect TechLab Ltd.’s employees to visit us in a one year period if we accept their offer? 

In [29]:
df1.Payor.unique()

array(['Cash', 'CIC Corporate', 'CIC Photocard', 'Resolution', 'Madison',
       'AON', 'Britam', 'Sunshine Ltd.', 'UAP', 'Jubilee'], dtype=object)

In [30]:
sunshine_count = df1[df1.Payor == "Sunshine Ltd."]

In [31]:
len(sunshine_count)

226

In [32]:
(226/200)*100

112.99999999999999

In [33]:
(113/100) * 500

565.0

### How much profit or loss do you expect Penda would make should Penda accept Techlab Ltd.’s offer? (If you expect a loss, express it as a negative integer.)

In [34]:
750000 - (55/100 * (565*1054.7197958323518))

422245.82344509667

### Using the data provided, estimate the total cost to Penda for the visits from TechLab Ltd. over a one year period. Assume that all non-capitation visits have a gross margin of 45%, and that capitation visits access a similar range of services – in other words, assume that capitation visits cost Penda the same amount as non-capitation visits on average.

In [35]:
df1.InvoiceTotal.mean()

1054.7197958323518

In [36]:
55/100 * (565*1054.7197958323518)

327754.17655490333

One risk with capitation schemes is that we may experience cannibalization – in other words, some of the patients on the capitation scheme may have already been using Penda Health for medical services, and will now move over to the capitated scheme rather than paying us for services accessed. 

For example, if patient with unique PatientID PAT123456 previously payed cash for medical services, and visited Penda Health on average twice a year, with an average bill size of Sh700, and this patient is now placed on a capitated scheme by their employer, they will now have the option to visit Penda Health as many times as they would like without paying at all. 

Given the above, it is important for us to understand the effect of cannibalization when we consider capitation agreements.

Remember to round down all answers to the nearest integer

### Qn. How many Sunshine Ltd. patients also visited Penda Health under a different payor? Give the number of unique patients, not the number of visits. 

##### Find the Patients who visited Penda Health under Sunshine Ltd.

In [37]:
#Under Sunshine Payor
sunshine_count = df1[df1.Payor == "Sunshine Ltd."]

##### Filter to find which PatientId under a different payor matches the one under the Sunshine payor

In [38]:
#Other Payor Excluding Sunshine Ltd
other_payor = df1[df1.Payor.isin(["Cash","CIC Corporate",'CIC Photocard','Resolution','Madison','AON',
                    'Britam','UAP','Jubilee'])]

In [39]:
# Find the columns with matching patientId To find the total number of visits
'''
merged_df = pd.DatFrame()
for i in sunshine_count.PatientId:
    for j in other_payor.PatientId:
        if i == j:
            merged_id.append(j)
len(patientId)
'''
df_merged = sunshine_count.merge(
    other_payor,
    on='PatientId',
    suffixes=('_sunshine_count', '')
    )
print(len(df_merged))

229


In [40]:
# Find the unique PatientId's from the dataset. 
'''
num_of_unque_patients = []
for i in patientId:
    if i not in num_of_unque_patients:
        num_of_unque_patients.append(i)
len(num_of_unque_patients)
'''
print(len(df_merged.PatientId.unique()))

40


The Number of Patients is: 40

### If the non-Sunshine Ltd. visits for these patients had been billed under Sunshine Ltd. instead, how much revenue would Penda have lost? *

In [41]:
# df1
df_merged.InvoiceTotal.sum()

219196.9

### If you assume that 50% of visits from Sunshine Ltd. in 2019 would have occurred anyway even if Penda had not accepted the capitation agreement with Sunshine Ltd., and that these visits would have had an average invoice amount equal to that of cash visits, how much revenue Penda would have made from these visits?

In [42]:
#Under Sunshine Payor
sunshine_count = df1[df1.Payor == "Sunshine Ltd."]

In [43]:
# Average of Cash payments 
total_Cash_count = df1[df1.Payor == "Cash"]

In [44]:
total_Cash_count.InvoiceTotal.mean()

617.0608034046772

In [45]:
50/100 * len(sunshine_count) 

113.0

In [46]:
113.0 * (total_Cash_count.InvoiceTotal.mean())

69727.87078472853

### What was the gross margin for the Sunshine Ltd. scheme in 2019? Again, assume that capitation visits utilize similar services to non-capitation visits (and so have the same average cost), and that non-capitation visits have a gross margin of 45%.

In [47]:
sunshine_count.InvoiceTotal.sum()

1540.0

In [48]:
len(sunshine_count)

226

In [49]:
45/100 *(1540 * 226)

156618.0